In [1]:
import pandas as pd
from tqdm.auto import tqdm

/mnt/c/Users/KonuTech/llm-zoomcamp/llm-zoomcamp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np

# Set numpy to suppress scientific notation and control precision
np.set_printoptions(suppress=True, precision=6)

# Test array
arr = np.array([1.23456789e-10, 1.23456789e-5, 1.23456789e-2])
print("Numpy Array:")
print(arr)


Numpy Array:
[0.       0.000012 0.012346]


In [3]:
import os

In [4]:
model_name= "multi-qa-mpnet-base-dot-v1"

In [5]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [6]:
os.getcwd()

'/mnt/c/Users/KonuTech/llm-zoomcamp/llm-zoomcamp/04-monitoring'

In [7]:
results = '/mnt/c/Users/KonuTech/llm-zoomcamp/llm-zoomcamp/04-monitoring/data/results-gpt4o-mini.csv'

In [8]:
df = pd.read_csv(results)

In [9]:
# df[:300]

# Q1:

In [10]:
answer_llm = df.iloc[0].answer_llm

In [11]:
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [12]:
model.encode(answer_llm)

array([-0.015224, -0.077747, -0.009609,  0.0102  , -0.024744,  0.033753,
       -0.09562 , -0.030484, -0.105333,  0.003071,  0.046147, -0.006697,
        0.012337, -0.007615,  0.044528, -0.012739,  0.01017 , -0.01208 ,
        0.043755, -0.055053, -0.010847, -0.028431, -0.032893,  0.024828,
        0.022821, -0.03023 ,  0.019501,  0.056183, -0.021423, -0.038149,
       -0.015333, -0.012188,  0.065007,  0.020174,  0.057469,  0.063441,
        0.025594,  0.002886, -0.063613, -0.012482, -0.016234, -0.032075,
        0.062827,  0.051422,  0.060449,  0.042878, -0.007704, -0.038713,
        0.080526,  0.068665,  0.002594, -0.072757,  0.002756, -0.028536,
       -0.05333 ,  0.070403,  0.026196,  0.031028, -0.038723,  0.012589,
       -0.099994, -0.09504 , -0.083946,  0.047963, -0.045295,  0.042159,
       -0.03426 ,  0.125731,  0.104514, -0.095115, -0.002561, -0.051539,
       -0.023379,  0.058121,  0.069549,  0.036254, -0.012421,  0.03648 ,
        0.106596,  0.0599  , -0.005986,  0.034055, 

# Q2:

In [13]:
!head data/results-gpt4o-mini.csv

answer_llm,answer_orig,document,question,course
You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).,"Machine Learning Zoomcamp FAQ
The purpose of this document is to capture frequently asked technical questions.
We did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:
Data Engineering Zoomcamp FAQ
In the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo
work",0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
You can sign up using the link provided in the course GitHub repository: [https://airtable.com/shryxwLd0COOEaqXo](https://airtable.com/shryxwLd0COOEaqXo).,"Machine Learning Zoomcamp FAQ
The purpose of this document is to capture frequently asked technical questions.
We did this for our data engineering course and it worked quite well. Check this document for inspira

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
results_gpt4o_mini = df.to_dict(orient='records')
# results_gpt4o_mini

In [15]:
record = results_gpt4o_mini[0]
record

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [16]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = model.encode(answer_llm)
    v_orig = model.encode(answer_orig)
    
    return v_llm.dot(v_orig)

In [17]:
similarity = []

for record in tqdm(results_gpt4o_mini):
    sim = compute_similarity(record)
    similarity.append(sim)

100%|███████████████████████████| 1830/1830 [00:29<00:00, 62.67it/s]


In [18]:
df['cosine'] = similarity
df['cosine'].describe()

count    1830.000000
mean        0.680332
std         0.215962
min        -0.141910
25%         0.585866
50%         0.733998
75%         0.836750
max         0.982701
Name: cosine, dtype: float64

In [19]:
df

,answer_llm,answer_orig,document,question,course,cosine
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,0.388595
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,0.298360
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0.583048
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,-0.019481
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,0.358467
...,...,...,...,...,...,...
1825,Some suggested titles for listing the Machine ...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,What are some suggested titles for listing the...,machine-learning-zoomcamp,0.620892
1826,It is best advised that you do not list the Ma...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,Should I list the Machine Learning Zoomcamp ex...,machine-learning-zoomcamp,0.730319
1827,You can incorporate your Machine Learning Zoom...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,In which LinkedIn sections can I incorporate m...,machine-learning-zoomcamp,0.679815
1828,The advice on including a project link in a CV...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,Who gave advice on including a project link in...,machine-learning-zoomcamp,0.355928


# Q3:

In [20]:
import numpy as np

def compute_similarity_normalized(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    # Encode the answers to get the embeddings
    v_llm = model.encode(answer_llm)
    v_orig = model.encode(answer_orig)
    
    # Normalize the vectors
    norm_llm = np.sqrt((v_llm * v_llm).sum())
    norm_orig = np.sqrt((v_orig * v_orig).sum())
    
    v_llm_norm = v_llm / norm_llm
    v_orig_norm = v_orig / norm_orig
    
    # Compute the dot product of the normalized vectors
    similarity = v_llm_norm.dot(v_orig_norm)
    
    return similarity


In [21]:
import numpy as np

def compute_similarity_normalized(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    # Encode the answers to get the embeddings
    v_llm = model.encode(answer_llm)
    v_orig = model.encode(answer_orig)
    
    # Normalize the vectors
    norm_llm = np.sqrt((v_llm * v_llm).sum())
    norm_orig = np.sqrt((v_orig * v_orig).sum())
    
    v_llm_norm = v_llm / norm_llm
    v_orig_norm = v_orig / norm_orig
    
    # Compute the dot product of the normalized vectors
    cosine_similarity = v_llm_norm.dot(v_orig_norm)
    
    # Ensure the result is within [0, 1]
    similarity = (cosine_similarity + 1) / 2
    
    return similarity


In [22]:
similarity_normalized = []

for record in tqdm(results_gpt4o_mini):
    sim = compute_similarity_normalized(record)
    similarity_normalized.append(sim)

100%|███████████████████████████| 1830/1830 [00:31<00:00, 58.51it/s]


In [23]:
df['cosine_normalized'] = similarity_normalized
df['cosine_normalized'].describe()

count    1830.000000
mean        0.840166
std         0.107981
min         0.429045
25%         0.792933
50%         0.866999
75%         0.918375
max         0.991351
Name: cosine_normalized, dtype: float64

In [24]:
df

,answer_llm,answer_orig,document,question,course,cosine,cosine_normalized
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,0.388595,0.694297
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,0.298360,0.649180
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0.583048,0.791524
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,-0.019481,0.490260
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,0.358467,0.679233
...,...,...,...,...,...,...,...
1825,Some suggested titles for listing the Machine ...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,What are some suggested titles for listing the...,machine-learning-zoomcamp,0.620892,0.810446
1826,It is best advised that you do not list the Ma...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,Should I list the Machine Learning Zoomcamp ex...,machine-learning-zoomcamp,0.730319,0.865160
1827,You can incorporate your Machine Learning Zoom...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,In which LinkedIn sections can I incorporate m...,machine-learning-zoomcamp,0.679815,0.839907
1828,The advice on including a project link in a CV...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,Who gave advice on including a project link in...,machine-learning-zoomcamp,0.355928,0.677964


# Q4:

In [25]:
df[df["document"]=="5170565b"]

,answer_llm,answer_orig,document,question,course,cosine,cosine_normalized
10,"Yes, all sessions are recorded, so if you miss...","Everything is recorded, so you won’t miss anyt...",5170565b,Are sessions recorded if I miss one?,machine-learning-zoomcamp,0.684030,0.842015
11,"Yes, you can ask your questions in advance if ...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I ask questions in advance if I can't atte...,machine-learning-zoomcamp,0.723019,0.861510
12,"If you miss a session, don't worry! Everything...","Everything is recorded, so you won’t miss anyt...",5170565b,How will my questions be addressed if I miss a...,machine-learning-zoomcamp,0.913276,0.956638
13,"Yes, there is a way to catch up on a missed se...","Everything is recorded, so you won’t miss anyt...",5170565b,Is there a way to catch up on a missed session?,machine-learning-zoomcamp,0.795638,0.897819
14,"Yes, you can still interact with instructors a...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I still interact with instructors after mi...,machine-learning-zoomcamp,0.615270,0.807635


In [26]:
df.iloc[10:11]

,answer_llm,answer_orig,document,question,course,cosine,cosine_normalized
10,"Yes, all sessions are recorded, so if you miss...","Everything is recorded, so you won’t miss anyt...",5170565b,Are sessions recorded if I miss one?,machine-learning-zoomcamp,0.68403,0.842015


In [27]:
df["answer_llm"].iloc[10:11]

10    Yes, all sessions are recorded, so if you miss...
Name: answer_llm, dtype: object

In [28]:
df["answer_orig"].iloc[10:11]

10    Everything is recorded, so you won’t miss anyt...
Name: answer_orig, dtype: object

In [29]:
from rouge import Rouge
rouge_scorer = Rouge()

# scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores = rouge_scorer.get_scores(df["answer_llm"].iloc[10:11], df["answer_orig"].iloc[10:11])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

# Q5:

In [30]:
scores["rouge-1"]["f"]

0.45454544954545456

In [31]:
scores["rouge-2"]["f"]

0.21621621121621637

In [32]:
scores["rouge-l"]["f"]

0.393939388939394

In [33]:
(scores["rouge-1"]["f"] + scores["rouge-2"]["f"] + scores["rouge-l"]["f"]) / 3

0.35490034990035496

# Q6:

In [34]:
import pandas as pd
from rouge import Rouge

# Assuming df is your DataFrame
rouge_scorer = Rouge()

# Initialize variables to accumulate the sum of F-scores
total_f_score_rouge1 = 0
total_f_score_rouge2 = 0
total_f_score_rougeL = 0

# Iterate over all rows in the DataFrame
for index in range(len(df)):
    # Get the LLM-generated answer and the original answer
    answer_llm = df["answer_llm"].iloc[index]
    answer_orig = df["answer_orig"].iloc[index]
    
    # Compute the Rouge scores
    scores = rouge_scorer.get_scores(answer_llm, answer_orig)[0]
    
    # Accumulate the F-scores
    total_f_score_rouge1 += scores["rouge-1"]["f"]
    total_f_score_rouge2 += scores["rouge-2"]["f"]
    total_f_score_rougeL += scores["rouge-l"]["f"]

# Calculate the average F-scores across all examples
average_f_score_rouge1 = total_f_score_rouge1 / len(df)
average_f_score_rouge2 = total_f_score_rouge2 / len(df)
average_f_score_rougeL = total_f_score_rougeL / len(df)

# Compute the overall average F-score across all rouge metrics
overall_average_f_score = (average_f_score_rouge1 + average_f_score_rouge2 + average_f_score_rougeL) / 3

# Output the results
print(f"Average F-score for Rouge-1: {average_f_score_rouge1}")
print(f"Average F-score for Rouge-2: {average_f_score_rouge2}")
print(f"Average F-score for Rouge-L: {average_f_score_rougeL}")
print(f"Overall Average F-score: {overall_average_f_score}")


Average F-score for Rouge-1: 0.3516946452113944
Average F-score for Rouge-2: 0.1767170469826221
Average F-score for Rouge-L: 0.32758565643306686
Overall Average F-score: 0.2853324495423611
